In [1]:
#### Import Section
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import env
import wrangle_zillow
import model

from os.path import exists
from itertools import product
from scipy.stats import levene , pearsonr, spearmanr, mannwhitneyu, f_oneway, ttest_ind
from sklearn.metrics import mean_squared_error, explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, TweedieRegressor, LassoLars
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import RFE, f_regression, SelectKBest

import warnings
warnings.filterwarnings("ignore")

# Model
Using scaled dataset

In [2]:
df, train, validate, test, train_scaled, validate_scaled, test_scaled  = wrangle_zillow.wrangle_zillow()

(28971, 23) (12417, 23) (10348, 23)
-----
DataFrame info:

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51736 entries, 1727539 to 1187175
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   basementsqft                51736 non-null  float64 
 1   bathrooms                   51736 non-null  float64 
 2   bedrooms                    51736 non-null  float64 
 3   area                        51736 non-null  float64 
 4   county                      51736 non-null  object  
 5   garagecarcnt                51736 non-null  float64 
 6   garagetotalsqft             51736 non-null  float64 
 7   latitude                    51736 non-null  float64 
 8   longitude                   51736 non-null  float64 
 9   lotsizesquarefeet           51736 non-null  float64 
 10  poolcnt                     51736 non-null  float64 
 11  structuretaxvaluedollarcnt  51736 non-null  float64 
 12  home_va

# Functions For Export

## Modeling Prep

In [352]:
def modeling_prep (train, train_scaled, validate, validate_scaled, test, test_scaled):
   """
   Purpose
      To return X, y subsets for training, validation, and testing of models

   Parameters
      train/validate/test: dataframes containing appropriate subsets of data
      train_scaled, validate_scaled, test_scaled: dataframes contianing scaled versions of approrpriate subsets of data

   Returns
      X_train, y_train, X_val, y_val, X_test, y_test: dataframes containing appropriate subsets of data
   """
   # create X,y for train, validate and test subsets
   X_train = train_scaled.drop(columns=['logerror','logerror_bin'])
   y_train = train.logerror
   X_val = validate_scaled.drop(columns=['logerror', 'logerror_bin'])
   y_val = validate.logerror
   X_test = test_scaled.drop(columns=['logerror', 'logerror_bin'])
   y_test = test.logerror

   #shift y subsets into a data frame
   y_train = pd.DataFrame(y_train)
   y_val = pd.DataFrame(y_val)
   y_test = pd.DataFrame(y_test)

   #add baseline predictions
   y_train['pred_median'] = y_train.logerror.median()
   y_val['pred_median'] = y_val.logerror.median()
   y_test['pred_median'] = y_test.logerror.median()

   #get dummies for X subsets
   dummy_columns = ['county',
               'poolcnt',
               'home_size',
               'aircon',
               'heating',
               'cluster house_tax',
               'cluster house_details',
               'cluster house_sizing',
               'cluster house_locale',
               ]
   X_train = pd.get_dummies(X_train, columns=dummy_columns, drop_first=True)
   X_val = pd.get_dummies(X_val, columns=dummy_columns, drop_first=True)
   X_test = pd.get_dummies(X_test, columns=dummy_columns, drop_first=True)

   return X_train, y_train, X_val, y_val, X_test, y_test

In [353]:
X_train, y_train, X_val, y_val, X_test, y_test = modeling_prep(train, train_scaled, validate, validate_scaled, test, test_scaled)

## Select K Best

In [5]:
def select_kbest(X, y, k): 
    """
    Purpose
        To return the top features selecting by the SelectKBest function

    Parameters
       X: dataframe containing X subset of features for the data subset
       y: dataframe with series containing the target variable
       k: the number of features for the function to suggest 
    Returns
       f_top_features: list of the top features selected by SelectKBest function
    """
    # initilize selector object
    f_selector = SelectKBest(f_regression, k=k)

    #fit object --> will find top 2 as requested
    f_selector.fit(X, y)

    # create mask
    feature_mask = f_selector.get_support()

    # use mask to show list of feature support
    f_top_features = X.iloc[:,feature_mask].columns.tolist()

    return f_top_features

## RFE

In [6]:
def rfe (X, y, n):
    """
    Purpose
        To return the top features selected by the RFE function

    Parameters
       X: dataframe containing X subset of features for the data subset
       y: dataframe with series containing the target variable
       n: the number of features for the function to select 
    Returns
       rfe_tip_features: list of the top features selected by SelectKBest function
    """
    #initialize  regression object
    lm = LinearRegression()

    # initilize RFE object with n features
    rfe = RFE(lm, n_features_to_select=n)

    #fit object onto data
    rfe.fit(X, y)

    #create boolean mask for columns model selects 
    feature_mask = rfe.support_

    # use mask to show list of selected features
    rfe_top_features = X.iloc[:, feature_mask].columns.tolist()

    return rfe_top_features


## Get Features

In [383]:
def get_features(X_train, y_train):
    """
    Purpose
        create a list of feature combinations to feed into the various models

    Parameters
       X_train: dataframe containing X subset of features for the data subset
       y_train: dataframe with series containing the target variable
       
    Returns
       feat_combos: list feature combinations
    """
    #create lists of features

    tax_feat = ['cluster house_tax_0.25', 'cluster house_tax_0.5', 'cluster house_tax_0.75', 'cluster house_tax_1.0',
                  'cluster house_locale_0.25', 'cluster house_locale_0.5', 'cluster house_locale_0.75', 'cluster house_locale_1.0',
                  'cluster house_sizing_0.25', 'cluster house_sizing_0.5', 'cluster house_sizing_0.75', 'cluster house_sizing_1.0',
                  'area', 'poolcnt_1.0', 'home_value', 'county_Orange County', 'county_Ventura County']

    details_feat = ['cluster house_details_0.3333333333333333', 'cluster house_details_0.6666666666666666', 'cluster house_details_1.0',
                  'cluster house_tax_0.25', 'cluster house_tax_0.5', 'cluster house_tax_0.75', 'cluster house_tax_1.0',
                  'cluster house_locale_0.25', 'cluster house_locale_0.5', 'cluster house_locale_0.75', 'cluster house_locale_1.0',
                  'area', 'poolcnt_1.0', 'home_value', 'county_Orange County', 'county_Ventura County']

    sizing_feat = ['cluster house_sizing_0.25', 'cluster house_sizing_0.5', 'cluster house_sizing_0.75', 'cluster house_sizing_1.0',
                  'cluster house_locale_0.25', 'cluster house_locale_0.5', 'cluster house_locale_0.75', 'cluster house_locale_1.0',
                  'area', 'poolcnt_1.0', 'home_value', 'county_Orange County', 'county_Ventura County']

    locale_feat = ['cluster house_locale_0.25', 'cluster house_locale_0.5', 'cluster house_locale_0.75', 'cluster house_locale_1.0',
                  'cluster house_details_0.3333333333333333', 'cluster house_details_0.6666666666666666', 'cluster house_details_1.0',
                  'area', 'poolcnt_1.0', 'home_value', ]

    feat_rfe = rfe(X_train, y_train.logerror, 10)
    print(feat_rfe)
    feat_sk_best = select_kbest(X_train, y_train.logerror, 10)
    print(feat_sk_best)

    #combine lists of features into large list feature all selected combinations
    feat_combos = [tax_feat, details_feat, sizing_feat, locale_feat, feat_sk_best]

    return feat_combos

In [368]:
get_features(X_train, y_train)

['area', 'structuretaxvaluedollarcnt', 'home_value', 'landtaxvaluedollarcnt', 'tax_per_sqft', 'poolcnt_1.0', 'cluster house_details_0.3333333333333333', 'cluster house_details_0.6666666666666666']
['bathrooms', 'bedrooms', 'area', 'garagecarcnt', 'garagetotalsqft', 'county_Orange County', 'home_size_large', 'cluster house_locale_0.75']


[['cluster house_tax_0.25',
  'cluster house_tax_0.5',
  'cluster house_tax_0.75',
  'cluster house_tax_1.0',
  'county_Orange County',
  'county_Ventura County'],
 ['cluster house_details_0.3333333333333333',
  'cluster house_details_0.6666666666666666',
  'cluster house_details_1.0',
  'county_Orange County',
  'county_Ventura County'],
 ['cluster house_sizing_0.25',
  'cluster house_sizing_0.5',
  'cluster house_sizing_0.75',
  'cluster house_sizing_1.0',
  'cluster house_locale_0.25',
  'cluster house_locale_0.5',
  'cluster house_locale_0.75',
  'cluster house_locale_1.0',
  'county_Orange County',
  'county_Ventura County'],
 ['cluster house_locale_0.25',
  'cluster house_locale_0.5',
  'cluster house_locale_0.75',
  'cluster house_locale_1.0',
  'cluster house_details_0.3333333333333333',
  'cluster house_details_0.6666666666666666',
  'cluster house_details_1.0',
  'county_Orange County',
  'county_Ventura County'],
 ['bathrooms',
  'bedrooms',
  'area',
  'garagecarcnt',
  'ga

In [19]:
# feat_combos = get_features(X_train, y_train)

## PF Mod

In [152]:
def pf_mod(X, y, selectors, scores, fit_train=None, fit_y_train=None):
    """
    Purpose
       to create, train, and score linear regression models using polynomial features
    Parameters
       X: dataframe containing X subset of features for the data subset
       y: dataframe with series containing the target variable
       selectors: list of different feature and degree combinations for use with models
       fit_train: optional, X_train subset of data to fit the model when needing to score validation or test subsets
       fit_y_train: optional, X_train subset of data to fit the model when needing to score validation or test subsets
    Returns
       pf_description: dataFrame containing the scores, features, and parameters of the created models
    """
    #create empty data frame to hold model descriptions    
    #pf_descriptions = pd.DataFrame({}, columns=['Name','RMSE', 'Features', 'Parameters'])
    
    #loop through selector combinations to pull out different features and degree levels
    for idx, combo in enumerate(selectors):
        #create features object
        pf = PolynomialFeatures(degree=combo[1])
        #initialize model object
        lm = LinearRegression(normalize=True)
        #fit object on X_train subset depeneding on its position as parameter or the optional variant
        if fit_train is not None:
            fit_pf = pf.fit_transform(fit_train[combo[0]])
            X_pf = pf.transform(X[combo[0]])  
            lm.fit(fit_pf, fit_y_train.logerror)
        else:
            X_pf = pf.fit_transform(X[combo[0]])
            lm.fit(X_pf, y.logerror)

        model_label = f'Polynomial_{idx+1}'

        #predict
        if model_label in y:
            model_label = f'Polynomial_{randint(50,100)}'
            y[model_label] = lm.predict(X_pf)
        else:
            y[model_label] = lm.predict(X_pf)
         
        #calculate train rmse
        rmse = mean_squared_error(y.logerror, y[model_label], squared=False)

        description = pd.DataFrame([[model_label, rmse, combo[0], f'Degree: {combo[1]}']], columns=['Name', 'RMSE', 'Features', 'Parameters'])
        #pf_descriptions = pd.concat([pf_descriptions, description])
        scores = pd.concat([scores, description], ignore_index=True)

    return scores, y

In [309]:
# Notebook testing section
# #create a list of parameters
# pf_parameters = [2,3,4]

# #use list with product to create tuples of feature/parameter combination to feed into model
# pf_selectors = list(product(feat_combos, pf_parameters))

# #call pf function to get predictions for the Polynomial models and add to the dataframe
# pf_scores = pf_mod(X_train, y_train, pf_selectors, pf_scores)

# pf_scores

## OLS Mod

In [261]:
def ols_mod(X, y, selectors, scores, fit_x_train=None, fit_y_train=None):
    """
    Purpose
       to create, train, and score ordinary least squares linear regression modelss
    Parameters
       X: dataframe containing X subset of features for the data subset
       y: dataframe with series containing the target variable
       selectors: list of different feature and degree combinations for use with models
       fit_train: optional, X_train subset of data to fit the model when needing to score validation or test subsets
       fit_y_train: optional, X_train subset of data to fit the model when needing to score validation or test subsets
    Returns
       pf_description: dataFrame containing the scores, features, and parameters of the created models
    """

    #loop through selector combinations to pull out different features and degree levels
    for idx, features in enumerate(selectors):  
        #create model object
        lm = LinearRegression()
        
        #fit object on X_train subset depeneding on its position as parameter or the optional variant
        if fit_x_train is not None:
            lm.fit(fit_x_train[features], fit_y_train.logerror)
        else:   
            lm.fit(X[features], y.logerror)

        #create mdoel label
        model_label = f'OLS_{idx+1}'

        if model_label in y:
            model_label = f'OLS_{randint(50,100)}'
            y[model_label] = lm.predict(X[features])
        else:
            y[model_label] = lm.predict(X[features]) 

        #calc trian rmse
        rmse = mean_squared_error(y.logerror, y[model_label], squared=False)

        description = pd.DataFrame([[model_label, rmse, features, 'N/A']], columns=['Name', 'RMSE', 'Features', 'Parameters'])
        scores = pd.concat([scores, description], ignore_index=True)

    return scores, y

## Lars Mod

In [155]:
def lars_mod(X, y, selectors, scores, fit_x_train=None, fit_y_train=None):
   """
   Purpose
      to create, train, and score linear regression models using polynomial features
   Parameters
      X: dataframe containing X subset of features for the data subset
      y: dataframe with series containing the target variable
      selectors: list of different feature and degree combinations for use with models
      fit_train: optional, X_train subset of data to fit the model when needing to score validation or test subsets
      fit_y_train: optional, X_train subset of data to fit the model when needing to score validation or test subsets
   Returns
      pf_description: dataFrame containing the scores, features, and parameters of the created models
   """

   #create empty data frame to hold model descriptions    
   lars_descriptions = pd.DataFrame({}, columns=['Name','RMSE', 'Features', 'Parameters'])

   #loop through selector combinations to pull out different features and degree levels
   for idx, selector in enumerate(selectors):  
      #create model object
      lars = LassoLars(alpha=selector)
      #create mdoel label
      model_label = f'LARS_{idx+1}'

      if fit_x_train is not None:
         lars.fit(fit_x_train, fit_y_train.logerror)
      else:   
         lars.fit(X, y.logerror)

      #predict train
      if model_label in y:
         model_label = f'LARS_{randint(50,100)}'
         y[model_label] = lars.predict(X)
      else:
         y[model_label] = lars.predict(X) 

      #calc trian rmse
      rmse = mean_squared_error(y.logerror, y[model_label], squared=False)

      description = pd.DataFrame([[model_label, rmse, 'all', f'Alpha: {selector}']], columns=['Name', 'RMSE', 'Features', 'Parameters'])
      scores = pd.concat([scores, description], ignore_index=True)

   return scores, y



## GLM Mod

In [159]:
def GLM_mod(X, y, selectors, scores, fit_x_train=None, fit_y_train=None):
   """
   Purpose
      to create, train, and score linear regression models using polynomial features
   Parameters
      X: dataframe containing X subset of features for the data subset
      y: dataframe with series containing the target variable
      selectors: list of different feature and degree combinations for use with models
      fit_train: optional, X_train subset of data to fit the model when needing to score validation or test subsets
      fit_y_train: optional, X_train subset of data to fit the model when needing to score validation or test subsets
   Returns
      pf_description: dataFrame containing the scores, features, and parameters of the created models
   """
   
   #create empty data frame to hold model descriptions    
   glm_descriptions = pd.DataFrame({}, columns=['Name','RMSE', 'Features', 'Parameters'])

   #create empty data frame to hold model descriptions    
   for idx, combo in enumerate(selectors):  
      #create model object
      glm = TweedieRegressor(power=combo[0], alpha=combo[1])

      #create model label
      model_label = f'GLM_{idx+1}'

      #fit mode 
      #glm.fit(X, y.logerror)
      #fit object on X_train subset depeneding on its position as parameter or the optional variant
      if fit_x_train is not None:
         glm.fit(fit_x_train, fit_y_train.logerror)
      else:   
         glm.fit(X, y.logerror)

      #predict train
      if model_label in y:
         model_label = f'GLM_{randint(50,100)}'
         y[model_label] = glm.predict(X)
      else:
         y[model_label] = glm.predict(X)
         
      #calc rmse
      rmse = mean_squared_error(y.logerror, y[model_label], squared=False)

      description = pd.DataFrame([[model_label, rmse, '-', f'Power,Alpha: {combo}']], columns=['Name', 'RMSE', 'Features', 'Parameters'])
      scores = pd.concat([scores, description], ignore_index=True)

   return scores, y 

In [16]:
#Notebook Testing Section
# Contains code to test modeling functions separately from each other

# #run ols model with feature combinations
# olf_descriptions = ols_mod(X_train, y_train, feat_combos)
# olf_descriptions
# #create a list of parameters
# lars_parameters = [.1, .2, .5, .6, .75, .9, 1]

# lars_descriptions = lars_mod(X_train, y_train, lars_parameters)
# lars_descriptions
# #create a list of parameters
# glm_parameters = [(0,0), (0,.25), (0,.5), (0,.75), (0,1)]

# glm_descriptions = GLM_mod(X_train, y_train, glm_parameters)

# glm_descriptions

# Train and Score

In [389]:
def score_on_train(X_train, y_train): 
   """
   Purpose
      to create, train, and score linear regression models using diffent feature sets
   Parameters
      X: dataframe containing X subset of features for the data subset
      y: dataframe with series containing the target variable
      selectors: list of different feature and degree combinations for use with models
      fit_train: optional, X_train subset of data to fit the model when needing to score validation or test subsets
      fit_y_train: optional, X_train subset of data to fit the model when needing to score validation or test subsets
   Returns
      pf_description: dataFrame containing the scores, features, and parameters of the created models
   """
   y_train = y_train[['logerror', 'pred_median']]

   #calc rmse
   rmse = mean_squared_error(y_train.logerror, y_train.pred_median, squared=False)

   #create empty dataframe to hold model descriptions
   scores = pd.DataFrame([['pred_median', rmse, 0, 'N/A', 'N/A']], columns=['Name','RMSE', 'r^2 score','Features', 'Parameters'])
   
   #create lists of features
   feat_combos = get_features(X_train, y_train)

   #create a lists of parameters
   pf_parameters = [2]
   lars_parameters = [.1, .25, .5, .75, 1]
   glm_parameters = [(0,0), (0,.25), (0,.5), (0,.75), (0,1)]

   #use list with product to create tuples of feature/parameter combination to feed into model
   pf_selectors = list(product(feat_combos, pf_parameters))

   #run ols model with feature combinations
   scores, holder = pf_mod(X_train, y_train, pf_selectors, scores)
   scores, holder = ols_mod(X_train, y_train, feat_combos, scores)
   scores, holder = lars_mod(X_train, y_train, lars_parameters, scores)
   scores, holder = GLM_mod(X_train, y_train, glm_parameters, scores)


   for idx, model in enumerate(y_train.drop(columns='logerror').columns):
      scores.iat[(idx),2] = explained_variance_score(y_train['logerror'], y_train[model])

   return scores

In [390]:
train_scores = score_on_train(X_train, y_train)

['area', 'lotsizesquarefeet', 'structuretaxvaluedollarcnt', 'home_value', 'landtaxvaluedollarcnt', 'openness', 'tax_per_sqft', 'poolcnt_1.0', 'cluster house_details_0.3333333333333333', 'cluster house_details_0.6666666666666666']
['bathrooms', 'bedrooms', 'area', 'garagecarcnt', 'garagetotalsqft', 'longitude', 'county_Orange County', 'home_size_large', 'heating_Other', 'cluster house_locale_0.75']


In [385]:
train_scores.sort_values(by='RMSE')

,Name,RMSE,r^2 score,Features,Parameters
1,Polynomial_1,0.185103,0.015988,"[cluster house_tax_0.25, cluster house_tax_0.5...",Degree: 2
2,Polynomial_2,0.185325,0.013623,"[cluster house_details_0.3333333333333333, clu...",Degree: 2
3,Polynomial_3,0.185358,0.013271,"[cluster house_sizing_0.25, cluster house_sizi...",Degree: 2
4,Polynomial_4,0.185786,0.008714,"[cluster house_locale_0.25, cluster house_loca...",Degree: 2
16,GLM_1,0.185828,0.008264,-,"Power,Alpha: (0, 0)"
6,OLS_1,0.185940,0.007068,"[cluster house_tax_0.25, cluster house_tax_0.5...",N/A
8,OLS_3,0.185973,0.006712,"[cluster house_sizing_0.25, cluster house_sizi...",N/A
7,OLS_2,0.185976,0.006680,"[cluster house_details_0.3333333333333333, clu...",N/A
5,Polynomial_5,0.186023,0.006182,"[bathrooms, bedrooms, area, garagecarcnt, gara...",Degree: 2
9,OLS_4,0.186049,0.005902,"[cluster house_locale_0.25, cluster house_loca...",N/A


# Validate and Score

In [378]:
def score_on_validate(train_scores, X_val, y_val, X_train, y_train): 
   """
   Purpose

   Parameters
      X: dataframe containing X subset of features for the data subset
      y: dataframe with series containing the target variable
      selectors: list of different feature and degree combinations for use with models
      fit_train: optional, X_train subset of data to fit the model when needing to score validation or test subsets
      fit_y_train: optional, X_train subset of data to fit the model when needing to score validation or test subsets
   Returns
      pf_description: dataFrame containing the scores, features, and parameters of the created models
   """
   y_val = y_val[['logerror', 'pred_median']]
   
   #create new dataframe that is the top 10 (for and runs them)
   validate_scores = train_scores.copy(deep=True)
   validate_scores = validate_scores.set_index('Name')

   #calc rmse
   rmse = mean_squared_error(y_val.logerror, y_val.pred_median, squared=False)

   #create empty dataframe to hold model descriptions
   model_descriptions = pd.DataFrame([['pred_median', rmse, 0, 'N/A', 'N/A']], columns=['Name','RMSE', 'r^2 score','Features', 'Parameters'])
   
   for model in validate_scores.index:
      #empty list to hold feature combinations
      feat_combos = []
  
      #create a lists of parameters
      pf_selectors = []
      lars_parameters = []
      glm_parameters = []

      if model.startswith('Pol'):
         features = validate_scores.loc[model]['Features']
         degree = int(validate_scores.loc[model]['Parameters'][-1])
         pf_selectors.append((features, degree))
         model_descriptions, y_val = pf_mod(X_val, y_val, pf_selectors, model_descriptions, X_train, y_train)
      elif model.startswith('GLM'):
         pow_alpha = eval(validate_scores.loc[model]['Parameters'][13:])
         glm_parameters.append(pow_alpha) if pow_alpha not in glm_parameters else None
         model_descriptions, y_val = GLM_mod(X_val, y_val, glm_parameters, model_descriptions, X_train, y_train)
         glm_parameters = []   
      elif model.startswith('LARS'):
         alpha = eval(validate_scores.loc[model]['Parameters'][7:])
         lars_parameters.append(alpha) if alpha not in lars_parameters else None
         model_descriptions, y_val = lars_mod(X_val, y_val, lars_parameters, model_descriptions, X_train, y_train)
      elif model.startswith('OLS_'):
         feat_combos.append((validate_scores.loc[model]['Features']))
         model_descriptions, y_val = ols_mod(X_val, y_val, feat_combos, model_descriptions, X_train, y_train)
      
      model_descriptions.iat[-1, 0] = model
      
   for idx, model in enumerate(y_val.drop(columns='logerror').columns):
      model_descriptions.iat[idx,2] = explained_variance_score(y_val['logerror'], y_val[model])

   return model_descriptions, y_val

In [386]:
validate_scores, y_val = score_on_validate(train_scores, X_val, y_val, X_train, y_train)


In [388]:
round(validate_scores.sort_values(by='RMSE'), 10)

,Name,RMSE,r^2 score,Features,Parameters
16,GLM_1,1.656298e-01,1.971353e-03,-,"Power,Alpha: (0, 0)"
6,OLS_1,1.656514e-01,1.734242e-03,"[cluster house_tax_0.25, cluster house_tax_0.5...",N/A
17,GLM_2,1.656785e-01,1.436811e-03,-,"Power,Alpha: (0, 0.25)"
8,OLS_3,1.656903e-01,1.256772e-03,"[cluster house_sizing_0.25, cluster house_sizi...",N/A
18,GLM_3,1.656965e-01,1.216881e-03,-,"Power,Alpha: (0, 0.5)"
7,OLS_2,1.657045e-01,1.083796e-03,"[cluster house_details_0.3333333333333333, clu...",N/A
19,GLM_4,1.657078e-01,1.062007e-03,-,"Power,Alpha: (0, 0.75)"
9,OLS_4,1.657132e-01,9.615029e-04,"[cluster house_locale_0.25, cluster house_loca...",N/A
20,GLM_5,1.657170e-01,9.434177e-04,-,"Power,Alpha: (0, 1)"
10,OLS_5,1.657342e-01,7.512994e-04,"[bathrooms, bedrooms, area, garagecarcnt, gara...",N/A


# Takeaways
* During initial testing a lot of overfitting happening when using all features
* `locale_feat` is set of features focusing on the clusters created using location based selection
* further bolstered feature sets by including features taken from models that did not perform as well, but were not at the bottom of the pack
* models us feature clusters based on tax or tax value did not perform well compared to the rest
* used features that appeared from rfe and select_kbest feature selection tools to bolster feature sets
-----------
* After continued testing the polynomial perform worse with the more robust feature sets. highly suggestive of overfitting. Will be eliminating those of degree 3 or more from further experimentation. 
* LARS models appear to be similarly unproductive
-----------
* including the locale clustering as features boosted performance of models using features with tax clusters and sizing clusters. however, also added in area, pool count, home value, and county
* for next round of mass model testing will add further blends that are inclusive of the tax, sizing, and locale clusters
-----------
* Best Performing Model: GLM 1, Power: 0, Alpha: 0

# Test

In [339]:
validate_scores.sort_values(by='RMSE').head(1)[['Features', 'Parameters']]

,Features,Parameters
21,-,"Power,Alpha: (0, 0)"


In [409]:
#reset y_test variable to remove scores from previous model
y_test = y_test[['logerror', 'pred_median']]

#find rmse of baseline predictions 
test_pred_rmse = mean_squared_error(y_test.logerror, y_test.pred_median, squared=False)

#creates dataframe to hold model score on test set
test_score = pd.DataFrame([['pred_median', test_pred_rmse, 0, 'N/A', 'N/A']], columns=['Name','RMSE', 'r^2 score','Features', 'Parameters'])

#select parameters or features for final modeling on test set
#test_selectors = [(['bedrooms', 'area', 'county_Orange County', 'home_size_large'], 2)]
test_parameters = [(0,0)]

#fit and use the model that scored highest on validate set
#test_score, holder = pf_mod(X_test, y_test, test_selectors, test_score, X_train, y_train)
test_score, holder = GLM_mod(X_test, y_test, test_parameters, test_score, X_train, y_train)

#adds correct model name to data frame
test_score.iat[1, 0] = 'GLM_1'

#add r^2 score 
test_score.iat[1,2] = explained_variance_score(y_test['logerror'], y_test.iloc[:,2])

In [410]:
test_score

,Name,RMSE,r^2 score,Features,Parameters
0,pred_median,0.019024,0.000000,N/A,N/A
1,GLM_1,0.560851,-0.636532,-,"Power,Alpha: (0, 0)"


# Model Module Testing Section
* section to test if the functions are being imported correctly from our model.py module.
* important to test because we will be using the imports in our Final Report to cut down on code contained therein
* **IMPORTANT** : feature selection happens in the get_features function of the model.py module. Suggest testing various features sets in this notebook before hard coding them into the module function 

In [2]:
df, train, validate, test, train_scaled, validate_scaled, test_scaled  = wrangle_zillow.wrangle_zillow()

(28971, 22) (12417, 22) (10348, 22)
-----
DataFrame info:

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51736 entries, 1727539 to 1187175
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   basementsqft                51736 non-null  float64 
 1   bathrooms                   51736 non-null  float64 
 2   bedrooms                    51736 non-null  float64 
 3   area                        51736 non-null  float64 
 4   county                      51736 non-null  object  
 5   garagecarcnt                51736 non-null  float64 
 6   garagetotalsqft             51736 non-null  float64 
 7   latitude                    51736 non-null  float64 
 8   longitude                   51736 non-null  float64 
 9   lotsizesquarefeet           51736 non-null  float64 
 10  poolcnt                     51736 non-null  float64 
 11  structuretaxvaluedollarcnt  51736 non-null  float64 
 12  home_va

In [3]:
X_train, y_train, X_val, y_val, X_test, y_test = model.modeling_prep(train, train_scaled, validate, validate_scaled, test, test_scaled)

In [4]:
train_scores = model.score_on_train(X_train, y_train)

['area', 'lotsizesquarefeet', 'structuretaxvaluedollarcnt', 'home_value', 'landtaxvaluedollarcnt', 'openness', 'tax_per_sqft', 'poolcnt_1.0', 'cluster house_details_0.3333333333333333', 'cluster house_details_1.0']
['bathrooms', 'bedrooms', 'area', 'garagecarcnt', 'garagetotalsqft', 'county_Orange County', 'home_size_large', 'cluster house_sizing_0.25', 'cluster house_locale_0.5', 'cluster house_locale_0.75']


In [5]:
train_scores

,Name,RMSE,r^2 score,Features,Parameters
0,pred_median,0.187037,0.000000,N/A,N/A
1,Polynomial_1,0.185119,0.015816,"[cluster house_tax_0.25, cluster house_tax_0.5...",Degree: 2
2,Polynomial_2,0.185335,0.013521,"[cluster house_details_0.3333333333333333, clu...",Degree: 2
3,Polynomial_3,0.185357,0.013289,"[cluster house_sizing_0.25, cluster house_sizi...",Degree: 2
4,Polynomial_4,0.185845,0.008084,"[cluster house_locale_0.25, cluster house_loca...",Degree: 2
5,Polynomial_5,0.185807,0.008488,"[bathrooms, bedrooms, area, garagecarcnt, gara...",Degree: 2
6,OLS_1,0.185923,0.007248,"[cluster house_tax_0.25, cluster house_tax_0.5...",N/A
7,OLS_2,0.185965,0.006803,"[cluster house_details_0.3333333333333333, clu...",N/A
8,OLS_3,0.185972,0.006730,"[cluster house_sizing_0.25, cluster house_sizi...",N/A
9,OLS_4,0.186049,0.005902,"[cluster house_locale_0.25, cluster house_loca...",N/A


In [6]:
validate_scores, y_val = model.score_on_validate(train_scores, X_val, y_val, X_train, y_train)

In [7]:
validate_scores.sort_values(by='RMSE').head(1)

,Name,RMSE,r^2 score,Features,Parameters
6,OLS_1,0.165636,0.001923,"[cluster house_tax_0.25, cluster house_tax_0.5...",N/A


In [8]:
def score_on_test(X_test, y_test, X_train, y_train):
    #reset y_test variable to remove scores from previous model
    y_test = y_test[['logerror', 'pred_median']]

    #find rmse of baseline predictions 
    test_pred_rmse = mean_squared_error(y_test.logerror, y_test.pred_median, squared=False)

    #creates dataframe to hold model score on test set
    test_score = pd.DataFrame([['pred_median', test_pred_rmse, 0, 'N/A', 'N/A']], columns=['Name','RMSE', 'r^2 score','Features', 'Parameters'])

    #select parameters or features for final modeling on test set
    #test_selectors = [(['bedrooms', 'area', 'county_Orange County', 'home_size_large'], 2)]
    test_parameters = [(0,0)]

    #fit and use the model that scored highest on validate set
    #test_score, holder = pf_mod(X_test, y_test, test_selectors, test_score, X_train, y_train)
    test_score, holder = GLM_mod(X_test, y_test, test_parameters, test_score, X_train, y_train)

    #adds correct model name to data frame
    test_score.iat[1, 0] = 'GLM_1'

    #add r^2 score 
    test_score.iat[1,2] = explained_variance_score(y_test['logerror'], y_test.iloc[:,2])
    print(y_test.iloc[:,2])

    return test_score

In [9]:
model.score_on_test(X_test, y_test, X_train, y_train)

id
2181062    0.016073
2709432    0.059567
1440501    0.028581
2035409    0.024077
1561917    0.005898
             ...   
2507645    0.023197
932951     0.008164
943748    -0.005140
1965404    0.026054
2371483    0.006972
Name: GLM_1, Length: 10348, dtype: float64


,Name,RMSE,r^2 score,Features,Parameters
0,pred_median,0.153141,0.000000,N/A,N/A
1,GLM_1,0.152704,0.002098,-,"Power,Alpha: (0, 0)"
